## Pinot Noir Metabolomics Data Analysis
### Samples collected on orbitrap processed in MZmine. 

Pinot noir grapes grown in the Okanagan were exposed to smoke and compared to control grapes that were not exposed to smoke. Grapes were frozen until anlaysis by positive and negative mode LCMS on an Orbitrap mass spectrometer. Full scan data was also collected using GCMS. 
LCMS data was processed in MZmine3, GCMS data was processed in MS-DIAL. 

Import libraries to be used

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# from sklearn.model_selection import train_test_split
import cimcb_lite as cb # statitiscal package from the Center for integrative metabolomics and computational biology for non-targeted MS data

In [2]:
# Merge peak Data with Metadata to align treatment information
# pnMetaData = pd.read_excel("C:/Users/7820/Desktop/Zandberg Grape Metab/PinotNoirSH/PinotNoirMetaData.xlsx")
# Datatemp = pd.read_excel("C:/Users/7820/Desktop/Zandberg Grape Metab/PinotNoirSH/Negative Mode/PinotNoir_SH_Neg_PyInput.xlsx", sheet_name="Data")
# Datatemp.merge(pnMetaData, how="left", left_on="SampleID", right_on="LIMS").to_excel("tempfile.xlsx")
#manually moved sample info columns to the front then resaved the pytoninput file

Import data

In [2]:
input = 'C:/Users/7820/Desktop/Zandberg Grape Metab/PinotNoirSH/Negative Mode/PinotNoir_SH_Neg_PyInput.xlsx'
dataTable, peakTable = cb.utils.load_dataXL(input, DataSheet='Data', PeakSheet = 'PeakInfo')

Loadings PeakFile: PeakInfo
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 34 TOTAL PEAKS: 2744
Done!


Calculate the RSD of each compound in the QC injections in order to filter dataset by compounds with inconsistent signals in QC samples. 

In [29]:
QC_means = dataTable[dataTable['SampleType'] == 'qc'].iloc[:, 3:].agg(['mean', 'std']).T
QC_means['QC_RSD'] = QC_means['std'] / QC_means['mean'] * 100
peakTable = peakTable.merge(QC_means.reset_index().rename(columns={'index': 'Name'}).drop(['mean', 'std'], axis=1), on='Name', how='left').fillna({'QC_RSD': 100})
peakTable.head()

,Idx,Name,mz,rt,Label,compound_name,mol_formula,neutral_mass,database_match_info,QC_RSD_x,QC_RSD_y
0,1,feature_4,179.0552,1.09,D-Glucose,D-Glucose,C6H12O6,180.0634,KEGG C00031,198.347721,198.347721
1,2,feature_7,226.9651,1.11,NaN,NaN,NaN,NaN,NaN,100.000000,NaN
2,3,feature_8,161.0443,1.11,Lichenin,Lichenin,C6H10O5,162.0528,KEGG C00478,15.895904,15.895904
3,4,feature_9,130.9822,1.12,NaN,NaN,NaN,NaN,NaN,4.827034,4.827034
4,5,feature_12,159.9781,1.13,"3,4-Dichloroaniline","3,4-Dichloroaniline",C6H5Cl2N,160.9799,KEGG C02791,3.328703,3.328703


Remove features wtih a QC_RSD higher than 20%. This number can be adjusted. 

In [5]:
rsd = peakTable['QC_RSD']
peakTableClean = peakTable[(rsd < 20)]
print('Started with {}'.format(len(peakTable)), 'now have {}'.format(len(peakTableClean)), 'peaks')

Started with 2744 now have 1200 peaks


Create PCA plots of the data. 


In [8]:
peaklist = peakTableClean['Name']                   # Set peaklist to the metabolite names in the peakTableClean
X = dataTable[peaklist].values                      # Extract X matrix from dataTable using peaklist
Xlog = np.log10(X)                                  # Log scale (base-10)
Xscale = cb.utils.scale(Xlog, method='auto')        # methods include auto, pareto, vast, and level
Xknn = cb.utils.knnimpute(Xscale, k=3)              # missing value imputation (knn - 3 nearest neighbors)

print("Xknn: {} rows & {} columns".format(*Xknn.shape))

cb.plot.pca(Xknn,
            pcx=1,                                                  # pc for x-axis
            pcy=2,                                                  # pc for y-axis
            group_label=dataTable['SampleType'])  

Xknn: 34 rows & 1200 columns


Loading BokehJS ...

Conduct univariate Statistics for each feature to determine which would be of interest for pathway analysis and identification confirmation.

In [9]:
dataTable_stats=dataTable[dataTable['Class'].isin(['smoke','control'])] #remove QC samples to do statistics between smoked and non-smoked
pos_outcome='smoke'
statsTable = cb.utils.univariate_2class(dataTable_stats, peakTableClean, group ='Class', posclass=pos_outcome, parametric=True)
statsTable.to_excel('statsCompoundTablePNpos.xlsx')
display(statsTable)

,Idx,Name,Label,Grp0_Mean,Grp0_Mean-95CI,Grp1_Mean,Grp1_Mean-95CI,Sign,TTestStat,TTestPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
1,3,feature_8,Lichenin,14973.994125,"(12586.69, 17361.3)",13981.036925,"(10882.75, 17079.32)",0,0.497578,0.626505,NaN,13,44.828,46.667,42.857,0.925662,0.207939,1.129694,0.305831
2,4,feature_9,NaN,21529.507364,"(20107.52, 22951.5)",23784.629385,"(21960.02, 25609.24)",1,-1.858072,0.076593,NaN,5,17.241,26.667,7.143,0.945349,0.214378,1.464142,0.239114
3,5,feature_12,"3,4-Dichloroaniline",136310.605667,"(128486.28, 144134.94)",153927.207143,"(143639.89, 164214.53)",1,-2.694031,0.011989,NaN,0,0.000,0.000,0.000,0.954670,0.241498,0.779778,0.385004
4,6,feature_13,Trichloroethanol glucuronide,7730.000800,"(6900.49, 8559.51)",8750.084600,"(7832.28, 9667.89)",1,-1.619420,0.133646,NaN,16,55.172,53.333,57.143,0.964721,0.824360,0.037143,0.850686
5,8,feature_26,NaN,33723.963600,"(31710.6, 35737.33)",40744.174286,"(37344.2, 44144.15)",1,-3.539228,0.001477,NaN,0,0.000,0.000,0.000,0.932177,0.062652,1.447666,0.239347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1196,2734,feature_5638,NaN,57198.911067,"(54107.8, 60290.02)",60200.428286,"(56922.91, 63477.94)",1,-1.306935,0.202257,NaN,0,0.000,0.000,0.000,0.949656,0.179317,0.093953,0.761562
1197,2736,feature_5642,NaN,61490.858733,"(57430.62, 65551.09)",70812.527500,"(63784.44, 77840.61)",1,-2.289437,0.030097,NaN,0,0.000,0.000,0.000,0.943505,0.123880,3.909849,0.058296
1198,2740,feature_5646,Ticlopidine,79860.686714,"(72401.5, 87319.87)",84481.041143,"(74196.3, 94765.78)",1,-0.712786,0.482327,NaN,1,3.448,6.667,0.000,0.977395,0.784098,0.348768,0.559910
1199,2742,feature_5648,D-Mannitol 1-phosphate,38441.184933,"(35177.11, 41705.26)",40396.577286,"(36561.45, 44231.71)",1,-0.764456,0.451225,NaN,0,0.000,0.000,0.000,0.968790,0.527358,0.287347,0.596315


Filter the statistics table by pvalue to see how many compounds were significantly different between groups under a ttest with a pvalue cut off of 0.05.

In [13]:
ttestsigTable = statsTable[statsTable['TTestPvalue'] < 0.05 ].sort_values(by='TTestPvalue')
missing_count = ttestsigTable['Label'].isnull().sum()
print('there are {} compounds significantly different between treatments under ttest.'.format(len(ttestsigTable)))
print("{} of those have putative KEGG annoations".format(len(ttestsigTable)-missing_count))
display(ttestsigTable)

there are 332 compounds significantly different between treatments under ttest.
226 of those have putative KEGG annoations


,Idx,Name,Label,Grp0_Mean,Grp0_Mean-95CI,Grp1_Mean,Grp1_Mean-95CI,Sign,TTestStat,TTestPvalue,bhQvalue,TotalMissing,PercTotalMissing,Grp0_Missing,Grp1_Missing,ShapiroW,ShapiroPvalue,LeveneW,LevenePvalue
425,684,feature_1304,Coformycin,44536.883867,"(38115.23, 50958.54)",7.148859e+04,"(62322.62, 80654.55)",1,-4.772834,0.000056,NaN,0,0.000,0.000,0.000,0.976994,0.757372,1.242547,0.274805
292,428,feature_798,NaN,946882.203333,"(875049.62, 1018714.78)",1.162969e+06,"(1108511.32, 1217425.78)",1,-4.648082,0.000078,NaN,0,0.000,0.000,0.000,0.978760,0.805836,0.713876,0.405585
630,1023,feature_2010,NaN,16095.622427,"(11673.5, 20517.75)",3.557970e+04,"(28260.81, 42898.59)",1,-4.536441,0.000106,NaN,0,0.000,0.000,0.000,0.924560,0.039842,4.157318,0.051355
532,860,feature_1694,NaN,10635.238960,"(9448.76, 11821.71)",1.451864e+04,"(13307.95, 15729.32)",1,-4.487887,0.000121,NaN,0,0.000,0.000,0.000,0.954816,0.243567,0.000073,0.993251
291,427,feature_797,Fumarate,776393.200000,"(712656.7, 840129.7)",9.626533e+05,"(912536.31, 1012770.19)",1,-4.459599,0.000130,NaN,0,0.000,0.000,0.000,0.953663,0.227583,0.525940,0.474558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,653,feature_1258,Bis(2-chloroethyl)ether,68406.560643,"(64669.71, 72143.41)",6.357805e+04,"(61337.27, 65818.83)",0,2.082001,0.048176,NaN,3,10.345,6.667,14.286,0.884019,0.007007,5.506002,0.027541
1140,2467,feature_5151,NaN,24768.040182,"(15890.98, 33645.1)",1.477369e+04,"(10492.97, 19054.42)",0,2.088952,0.048495,NaN,5,17.241,26.667,7.143,0.896624,0.018266,2.542647,0.125075
675,1088,feature_2170,2-(4-Chlorophenyl)-3-phenyl-3-(2-pyridinyl)acr...,6439.745827,"(5146.49, 7733.0)",8.044895e+03,"(7377.99, 8711.8)",1,-2.097040,0.049604,NaN,8,27.586,26.667,28.571,0.964756,0.616512,2.614282,0.122388
826,1367,feature_2789,NaN,22468.833667,"(17385.47, 27552.2)",1.641096e+04,"(13933.78, 18888.14)",0,2.052431,0.049938,NaN,0,0.000,0.000,0.000,0.912075,0.019296,7.779649,0.009568


Merge significant compound statistics with the compound information to be output for future reference. 

In [17]:
SigPeakInfo = ttestsigTable.merge(peakTable, on ='Name', how='left')
SigPeakInfo.to_excel("sigpeaks.xlsx")
SigPeakInfo.head()

,Idx_x,Name,Label_x,Grp0_Mean,Grp0_Mean-95CI,Grp1_Mean,Grp1_Mean-95CI,Sign,TTestStat,TTestPvalue,...,LevenePvalue,Idx_y,mz,rt,Label_y,compound_name,mol_formula,neutral_mass,database_match_info,QC_RSD
0,684,feature_1304,Coformycin,44536.883867,"(38115.23, 50958.54)",7.148859e+04,"(62322.62, 80654.55)",1,-4.772834,0.000056,...,0.274805,684,283.1031,3.44,Coformycin,Coformycin,C11H16N4O5,284.1121,KEGG C01677,10.436836
1,428,feature_798,NaN,946882.203333,"(875049.62, 1018714.78)",1.162969e+06,"(1108511.32, 1217425.78)",1,-4.648082,0.000078,...,0.405585,428,134.0160,1.44,NaN,NaN,NaN,NaN,NaN,2.610556
2,1023,feature_2010,NaN,16095.622427,"(11673.5, 20517.75)",3.557970e+04,"(28260.81, 42898.59)",1,-4.536441,0.000106,...,0.051355,1023,451.1241,15.99,NaN,NaN,NaN,NaN,NaN,16.665130
3,860,feature_1694,NaN,10635.238960,"(9448.76, 11821.71)",1.451864e+04,"(13307.95, 15729.32)",1,-4.487887,0.000121,...,0.993251,860,171.0655,12.06,NaN,NaN,NaN,NaN,NaN,18.968941
4,427,feature_797,Fumarate,776393.200000,"(712656.7, 840129.7)",9.626533e+05,"(912536.31, 1012770.19)",1,-4.459599,0.000130,...,0.474558,427,115.0021,1.44,Fumarate,Fumarate,C4H4O4,116.0110,KEGG C00122,5.031928


Subset the initial quant dataframe exported by MZmine by significant peaks to isolate quantitative information about peaks with a significant p.value.

In [28]:
SigCompoundList = list(ttestsigTable['Name']) 
list2 = ['Idx', 'SampleID', 'Label', 'Class', 'Year', 'Company/Management', 'Vineyard', 'Block']
SigCompoundList = list2 + SigCompoundList 
SigDataTable = dataTable[SigCompoundList]
SigDataTable.to_excel("SigDataTable.xlsx")

The compounds shown to have a significant difference were searched by parent mass and KEGG annotation in the hormonomics database. 

All KEGG id's of annotated compounds regardless of significance were searched in the KEGG pathway finder via MetaboAnalayst